In [ ]:
import requests                # Include HTTP Requests module
from bs4 import BeautifulSoup  # Include BS web scraping module
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey, select
from sqlalchemy.sql import and_, or_, not_

In [ ]:
engine = create_engine('sqlite:///HoopStat.db')
conn = engine.connect()
metadata = MetaData(bind=None)
games = Table('games', metadata, autoload = True, autoload_with = engine)

In [ ]:
def convertToNumber(s):
    #https://stackoverflow.com/questions/31701991/string-of-text-to-unique-integer-method
    return int.from_bytes(s.encode(), 'little')

def getNewBatch(n):
    stmt = select([games.c.id]).where(and_(not_(games.columns.id.like('error%')), games.columns.isProcessed == False))\
    .limit(n)
    results = conn.execute(stmt).fetchall()
    results = [value for value, in results]
    return results

def createPBP(playsRaw,team):
    df = pd.DataFrame()
    for i in playsRaw:
        t = i.find('td',attrs={'class':'time'}).get_text().strip()
        x = i.find('span',attrs={'class':'text'}).get_text().strip()
        sc = i.find('td', attrs={'class':'score'}).get_text().strip()
        scA = 0
        scH = 0
        if sc:
            scH = sc.split('-')[0]
            scA = sc.split('-')[1]
        if i.find('span', attrs={'class':'v-score'}):
            scA = i.find('span', attrs={'class':'v-score'}).get_text().strip()
        if i.find('span', attrs={'class':'h-score'}):
            scH = i.find('span', attrs={'class':'h-score'}).get_text().strip()
        df = pd.concat([df,pd.DataFrame([[t,x,scH,scA]],columns=['gameTime','action','scoreHome','scoreAway'])])
    df['team'] = team
    return df

def scrapePeriod(soup,prd,gameID):
    home = soup.find_all('tr', attrs = {'class':['row home','row home score-changed']})
    away = soup.find_all('tr', attrs = {'class':['row visitor','row visitor score-changed']})

    df = pd.DataFrame()
    
    df = pd.concat([df,createPBP(home,'Home')])
    df = pd.concat([df,createPBP(away,'Away')])
    
    df['action'] = df.action.str.replace('\n','')
    df['action'] = df.action.str.replace('.','')
    df['action'] = df.action.str.replace("'",'')
    df['player'] = df.action.str.extract('([^a-z]{2,})')
    
    #https://stackoverflow.com/questions/61235091/string-modification-on-pandas-dataframe-subset
    mask_to = ~df['action'].str.contains('TIMEOUT') # same as df.action.str.contains('TIME')==False
    df.loc[mask_to,'action'] = df.loc[mask_to,'action'].str.replace('([^a-z0-9\._]{2,})','')
    #df[df['action'].str.contains('TIMEOUT')==False]['action'] = df[df['action'].str.contains('TIMEOUT')==False].action.str.replace('([^a-z0-9\._]{2,})','')#.str.split()

    df = df.set_index('gameTime')
    df = df.sort_index(ascending=False)
    df['time'] = df.index
    try:
        df['duration'] = pd.to_datetime(df['time'].astype(str)).diff().dt.total_seconds().div(-60)
    except:
        df['duration'] = 0
    df['uniqueID'] = np.random.randint(1000000,size=len(df.index))
    df['uniqueID'] = df['uniqueID'].map(str)
    df['gameID'] = gameID
    df['period'] = prd
    df['id'] = df['gameID'] + df['period'] + df['uniqueID']
    df['duration'] = df['duration'].fillna(0).map(int)

    return df[['id','gameID','time','action','scoreHome','scoreAway','team','duration','player','period']]
                                                
def getPeriods(gameID):
    url = "https://d3hoops.prestosports.com/seasons/men/2019-20/boxscores/" + gameID + ".xml?view=plays"
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    prds = soup.find_all('table', attrs = {'role':'presentation'})
    df = pd.DataFrame()
    for i in prds:
        prd = i.find('span')['id']
        try:
            df = pd.concat([df,scrapePeriod(i,prd,gameID)])
        except Exception as e:
            print(e)
    return df

def processBatch(n):
    processed = []
    errored = []
    results = getNewBatch(n)
    n = 1
    z = len(results)

    allGames = pd.DataFrame()

    for i in results:
        try:
            allGames = pd.concat([allGames,getPeriods(i)])
            processed.append(i)
        except Exception as e:
            errored.append(i)
            raise(e)
        #print(str(n) + " / " + str(z) + " records processed.")
        n += 1
    print(str(len(errored)) + ' errors. ', str(len(processed)) + ' games processed.')
    return allGames, processed

def removeDupes(df):
    df = df.reset_index()
    df.id = df.groupby('id').id.apply(lambda n: n + (np.arange(len(n))+1).astype(str))
    df = df.reset_index()
    return df[['id','gameID','time','action','scoreHome','scoreAway','team','duration','player','period']]

def writeGamesToDB(gms, processed, engine, table):
    gms = removeDupes(gms)
    tbl = Table(table, metadata, autoload = True, autoload_with = engine)
    for r in processed:
        conn.execute(tbl.delete(tbl.c.gameid == r))

    gms.to_sql(table, con=engine, if_exists='append', index=False)

    for p in processed:
        stmt = games.update().where(games.c.id==p).values(isProcessed=True)
        conn.execute(stmt)

def runThrough(writeToDB,batchsize,engine):
    gamesDF = pd.DataFrame()
    processedGames = []

    gamesDF,processedGames = processBatch(batchsize)
    
    if writeToDB == 1:
        writeGamesToDB(gamesDF,processedGames,engine,'events')

#https://stackoverflow.com/questions/19472922/reading-external-sql-script-in-python/19473206
def executeScriptsFromFile(filename):
    # Open and read the file as a single buffer
    fd = open(filename, 'r')
    sqlFile = fd.read()
    fd.close()

    # all SQL commands (split on ';')
    sqlCommands = sqlFile.split(';')

    # Execute every command from the input file
    for command in sqlCommands:
        # This will skip and report errors
        # For example, if the tables do not yet exist, this will skip over
        # the DROP TABLE commands
        try:
            engine.execute(command)
        except OperationalError as msg:
            print("Command skipped: " + msg)
            #https://stackoverflow.com/questions/19472922/reading-external-sql-script-in-python/19473206

In [ ]:
n=1000
while n > 0:
    try:
        runThrough(1,25,engine)
    except Exception  as e:
        print(e)
    n -= 25
    print(n)

In [ ]:
stmt = select([games.c.id]).where(games.columns.isProcessed == True)
results = conn.execute(stmt).fetchall()
gameIDs = [value for value, in results]
len(gameIDs)

In [ ]:
conn.close()

In [ ]:
engine.execute('ALTER TABLE %s ADD COLUMN %s %s' % ('players', 'tgp', 'INTEGER'))
#stmt = engine.execute('SELECT * FROM events')
#stmt.fetchall()

In [ ]:
executeScriptsFromFile(r'C:\Users\scohendevries\Documents\DATA698\update_events_playScore.sql')

In [ ]:
conn.execute("UPDATE games SET isProcessed = 0")

In [ ]:
conn.execute("SELECT * FROM games").fetchall()

In [ ]:
gameid = '20200104_1j0e'
getPeriods(gameid)